In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Input, Dropout
from keras.layers import Dense

In [ ]:
import pandas as pd
data = pd.read_pickle("modelos.pkl")
data['modelo']=data['modelo'].apply(lambda x: x.split('/')[0])
data.shape
data.head(5)

In [ ]:
df = pd.read_pickle("modelos_ensemble.pkl")
print(df.shape)
df.drop(columns=['index'],inplace=True)
print(df)

In [ ]:
from sklearn.model_selection import train_test_split

train=df.iloc[:,1:6]
y=df['target']
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)

In [ ]:
y_train_tf =  tf.keras.utils.to_categorical(y_train, 3)
y_test_tf =  tf.keras.utils.to_categorical(y_test, 3)
print(y_train_tf,y_train)

In [ ]:
from sklearn.utils import class_weight
import numpy as np
import math

def create_class_weight(labels_dict,mu=0.15):
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    class_weight = dict()
    
    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0
    
    return class_weight

# random labels_dict
labels_dict = {0:len(y_train[y_train==-1]),1: len(y_train[y_train==0]),2:len(y_train[y_train==1])}

class_weight=create_class_weight(labels_dict)

In [ ]:
#This callback will stop the training when there is no improvement in the loss for the consecutive nº of epochs defined in patience parameter.
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# define model
t_model = Sequential()
t_model.add(Dense(25, input_dim=5, activation='relu'))
t_model.add(Dense(3, activation='softmax'))
t_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
t_model.summary()

In [ ]:
# fit model
history = t_model.fit(X_train, y_train_tf, validation_data=(X_test, y_test_tf), epochs=10, callbacks=[callback],class_weight=class_weight, verbose=1)
t_model.save('saved_model/t_model')

In [ ]:
t_model = tf.keras.models.load_model('saved_model/t_model')
# Check its architecture
t_model.summary()

In [ ]:
t_model_pred = t_model.predict(X_test)
print(t_model_pred)

In [ ]:
import numpy as np

t_model_predictions = np.argmax(t_model_pred, axis=1) # select the index with the highest probability

fx = (lambda x: {0:0, 1:1, 2:-1}[x])

t_model_predictions = np.array([fx(x) for x in t_model_predictions]) # map the index to the corresponding label

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, t_model_predictions))